In [ ]:
#r "nuget:Microsoft.ML"
#r "nuget:Microsoft.ML.FastTree"

Installed Packages Microsoft.ML, 1.6.0 Microsoft.ML.FastTree, 1.6.0

In [ ]:
using System;
using System.IO;
using Microsoft.ML;
using Microsoft.ML.Data;

In [ ]:
var trainDataPath = "../datasets/weatherAUS-train.csv";
var testDataPath = "../datasets/weatherAUS-test.csv";

In [ ]:
public class WeatherData
{   
    [LoadColumn(0, 12)]
    public float Features { get; set; }
    
    [LoadColumn(13)]
    public bool RainTomorrow { get; set; }
}

In [ ]:
var mlContext = new MLContext(seed: 0);

In [ ]:
var trainingDataView = mlContext.Data.LoadFromTextFile<WeatherData>(trainDataPath, hasHeader: true, separatorChar: ',');
var testDataView = mlContext.Data.LoadFromTextFile<WeatherData>(testDataPath, hasHeader: true, separatorChar: ',');

In [ ]:
var dataProcessPipeline = mlContext.Transforms.Concatenate("Features", nameof(WeatherData.Features))
                                              .Append(mlContext.Transforms.NormalizeMinMax("Features", "Features"))
                                              .AppendCacheCheckpoint(mlContext);

In [ ]:
var trainer = mlContext.BinaryClassification.Trainers.FastTree(labelColumnName: nameof(WeatherData.RainTomorrow), featureColumnName: "Features");

var trainingPipeline = dataProcessPipeline.Append(trainer);

In [ ]:
var trainedModel = trainingPipeline.Fit(trainingDataView);

In [ ]:
var predictions = trainedModel.Transform(testDataView);

In [ ]:
var metrics = mlContext.BinaryClassification.EvaluateNonCalibrated(data: predictions, labelColumnName: nameof(WeatherData.RainTomorrow), scoreColumnName: "Score");

In [ ]:
Console.WriteLine($"Accuracy: {metrics.Accuracy:P2}");
Console.WriteLine($"F1Score: {metrics.F1Score:P2}");
Console.WriteLine($"Area Under Curve: {metrics.AreaUnderRocCurve:P2}\n");
Console.WriteLine(metrics.ConfusionMatrix.GetFormattedConfusionTable());

Accuracy: 85,24%
F1Score: 61,39%
Area Under Curve: 88,20%

TEST POSITIVE RATIO:	0,2238 (5354,0/(5354,0+18564,0))
Confusion table
          ||======================
PREDICTED || positive | negative | Recall
TRUTH     ||======================
 positive ||    2.806 |    2.548 | 0,5241
 negative ||      982 |   17.582 | 0,9471
          ||======================
Precision ||   0,7408 |   0,8734 |

